# Coursework 1 - This needs a proper title

## I. Background

- Why this study is of interest and relevant

## II. Study Purpose

### A. Motivation and Objectives

### B. Research Questions

In [ ]:
# import libraries
import pandas as pd

## II. Research Methodology

- Describe that the study will be primary a NLP project and this is why the NLP preparation to follow.

### A. Data Source

- Describe why Hellopeter 
- A good explanation as to why this data source has been selected.
- Justify that this is a major source of reviews/complaints which the telecoms actively monitor and respond to. Note that Telkom issued a notice (https://www.hellopeter.com/telkom) that they are phasing it out and urge customers to contact them via twitter opr facebook.
- Bellow implement the class to load data from Hellopeter via api


### B. Ethics of Use

 ### C. Data Selection

 - In this section also describe provider selection (telkom, vodacom, cell c, mtn) and why these major ones only.
 - in a separate cell use class and save all data to csv
 - in a separate cell discuss the features that will be selected, then in the next cell drop columns not of interest.

### D. Data Preparation

Sections (a paragraph followed by code) for:

- remove empty and illegal values - have clear reasoning described for changes
- somehow validate the data, DataFrame.Info maybe as a start, need to think about this
- prepare text for NLP analysis

### E. Limitations

- Describe limitations, only hellopeter, but there are many other sources of social media like facebook and twitter. 
- Predominantly complaints
- Specific to telecoms in south africa, similar techniques will be applicable to other industries.
- why the dataset is appropriate

## III. Exploratory Data Analysis

1. Use Hellopeter to get reviews from 3 cellular providers.
2. Clean-up - for example remove empty reviews
3. Data Prep
4. Analysis
	- Word cloud
	- bar-chart of stars clustered by providers
	- rating counts per month - is the ratings seasonal
	- boxplot of word count for reviews - also interesting correlation between word count and review stars
	- maybe a grid per telecom showing reviews per month vs the number of replies